# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import json
import requests
import gmaps
import os

# Import API key
from api_config import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
cities_output_df = pd.read_csv("Cities_Output.csv")
cities_output_df.dropna()
cities_output_df.head()

,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,0,Rikitea,5,PF,1584336617,79,-23.12,-134.97,79.47,11.25
1,1,Qaanaaq,100,GL,1584336617,47,77.48,-69.36,-0.98,7.20
2,2,Nikolskoye,90,RU,1584336617,80,59.70,30.79,34.00,22.37
3,3,Vaini,75,TO,1584336618,88,-21.20,-175.20,82.40,10.29
4,4,Port Alfred,99,ZA,1584336618,98,-33.59,26.89,63.00,3.00


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Access maps with unique API key
gmaps.configure(api_key=g_key)

In [4]:
# Store latitude and longitude in locations
locations = cities_output_df[["Lat", "Lng"]]
weight = cities_output_df["Humidity"].astype(float)

In [5]:
# Plot Heatmap
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=weight, 
                                 dissipating=False, max_intensity=10,
                                 point_radius=1)

# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [13]:
cities_output_df['Hotel Name'] = ""
cities_output_df

,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
0,0,Rikitea,5,PF,1584336617,79,-23.12,-134.97,79.47,11.25,
1,1,Qaanaaq,100,GL,1584336617,47,77.48,-69.36,-0.98,7.20,
2,2,Nikolskoye,90,RU,1584336617,80,59.70,30.79,34.00,22.37,
3,3,Vaini,75,TO,1584336618,88,-21.20,-175.20,82.40,10.29,
4,4,Port Alfred,99,ZA,1584336618,98,-33.59,26.89,63.00,3.00,
...,...,...,...,...,...,...,...,...,...,...,...
521,521,Jizan,40,SA,1584336753,61,17.33,42.67,82.40,4.70,
522,522,Ampanihy,7,MG,1584336753,62,-24.70,44.75,80.92,3.51,
523,523,Riyadh,0,SA,1584336676,48,24.69,46.72,64.40,2.24,
524,524,Panama City,20,PA,1584336754,65,8.99,-79.52,80.60,5.82,


In [24]:
hotel_df = cities_output_df.loc[(cities_output_df['Max Temp'] > 70) &
                                (cities_output_df['Max Temp'] < 77) &
                                (cities_output_df['Wind Speed'] < 10) &
                                (cities_output_df['Cloudiness'] == 0),
                                :].copy()
                           
hotel_df.dropna()
hotel_df

,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
239,239,Ibrā’,0,OM,1584336681,46,22.69,58.53,73.24,6.46,
336,336,Stann Creek Town,0,BZ,1584336706,93,16.97,-88.23,73.00,3.00,
338,338,Ifanadiana,0,MG,1584336707,69,-21.30,47.63,75.87,1.21,
391,391,Presidente Epitácio,0,BR,1584336719,88,-21.76,-52.12,72.34,2.84,
501,501,Umluj,0,SA,1584336748,27,25.02,37.27,76.26,4.63,


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [25]:
# find the hotel within 5000 m of coordinates

params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key,
}

# use iterrows to iterate through pandas dataframe
for index, row in hotel_df.iterrows():

    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    lat = row['Lat']
    lng = row['Lng']
    
    params['location'] = f"{lat},{lng}"

    response = requests.get(base_url, params=params).json()
    
#    print(json.dumps(response, indent=4, sort_keys=True))
    
    try:
        hotel_df.loc[index, 'Hotel Name'] = response['results'][0]['name']
    except (IndexError):
        print('Missing field/result... skipping.')
    
hotel_df.head()

Missing field/result... skipping.


,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
239,239,Ibrā’,0,OM,1584336681,46,22.69,58.53,73.24,6.46,مزرعة راحة النفوس المنزفة
336,336,Stann Creek Town,0,BZ,1584336706,93,16.97,-88.23,73.00,3.00,Chaleanor Hotel
338,338,Ifanadiana,0,MG,1584336707,69,-21.30,47.63,75.87,1.21,
391,391,Presidente Epitácio,0,BR,1584336719,88,-21.76,-52.12,72.34,2.84,Hotel Itaverá
501,501,Umluj,0,SA,1584336748,27,25.02,37.27,76.26,4.63,Tanal Hotel Apartment


In [26]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [27]:
# Add marker layer ontop of heat map

# Customize the size of the figure
#figure_layout = {
#    'width': '400px',
#    'height': '300px',
#    'border': '1px solid black',
#    'padding': '1px',
#    'margin': '0 auto 0 auto'
#}

#fig = gmaps.figure(layout=figure_layout)
fig = gmaps.figure()

# Assign the marker layer to a variable
markers = gmaps.marker_layer(locations)
# Add the layer to the map
fig.add_layer(markers)

# Display Map
fig

Figure(layout=FigureLayout(height='420px'))